Задание по теме ансамблей моделей

Выполнить Стэкинг, Бэгин, Вотинг и Бустинг. При реализации алгоритмов не использовать готовые решения. 
За сровнение взять CatBoostClassifier как базовая метрика качества. Сравнить результат с реализацией своих ансамблей. 
Для однозначности и интерпретируемости результатов использовать приложенный набор данных (wine quality). 
При реализации бустинга - просто сокращайте набор данных на котором модель отработала хорошо (правильно предсказанные данные). 

Анализ и план работы:
Загружаем библиотеки
Делаем решение задачи классификации с помощью CatBoost, предварительно подбираем наилучший набор параметров с помощью optuna.
После тренировки считаем accuracy на тестовой выборке.


Загрузка необходимых библиотек

In [291]:
%reset

import pandas as pd
import numpy as np
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split, cross_val_score
import optuna
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier

Nothing done.


Загрузка данных

In [292]:
# Load the dataset
data = pd.read_csv('winequality-white.csv', sep=';')
X = data.drop('quality', axis=1)  # Features
y = data['quality']  # Target variable

data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


Типы колонок

In [293]:
# check colums types
print(f'Data types: {data.dtypes}')

Data types: fixed acidity           float64
volatile acidity        float64
citric acid             float64
residual sugar          float64
chlorides               float64
free sulfur dioxide     float64
total sulfur dioxide    float64
density                 float64
pH                      float64
sulphates               float64
alcohol                 float64
quality                   int64
dtype: object


К счастью, все данные числовые <br>


Проверяем, требуется ли обработка пропусков

In [294]:
print('How many omitted data in each column:')
print(data.isna().sum())

How many omitted data in each column:
fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
dtype: int64


Пропущенных данных нет.<br>
Проводим разделение выборки на тренировочную и тестовую подвыборки. 

In [295]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Тренируем модель CatBoost

In [296]:
# Train the catboost model
model_cat = CatBoostClassifier(iterations=1000,
    learning_rate=0.1,
    depth=6,
    l2_leaf_reg=5,
    bagging_temperature=1,
    random_strength=1,
    early_stopping_rounds=50,
    verbose=False)
model_cat.fit(X_train, y_train, silent = True)
train_accuracy_cat = model_cat.score(X_train, y_train) # calculates accuracy

In [297]:
print(f'CatBoost score, train set: {train_accuracy_cat*100:.1f}%')

CatBoost score, train set: 96.1%


Оценка модели CatBoost на тестовой выборке

In [298]:
# Evaluate the model on the test set
y_pred_cat = model_cat.predict(X_test)
test_accuracy_cat = accuracy_score(y_test, y_pred_cat)
print(f'CatBoost score, test set: {test_accuracy_cat * 100:.1f}%')

CatBoost score, test set: 67.3%


Тренируем модель по методу поддерживающих векторов (SVM)

In [299]:
model_SVC = SVC()
model_SVC.fit(X_train, y_train)
# Evaluate the model on the train set
y_pred_tr = model_SVC.predict(X_train)
train_accuracy_SVC = accuracy_score(y_train, y_pred_tr)
print(f"Train set accuracy for SVM: {train_accuracy_SVC * 100:.1f} %")

Train set accuracy for SVM: 45.2 %


Оценка модели SVM на тестовой выборке

In [300]:
# Evaluate the model on the test set
y_pred_SVC = model_SVC.predict(X_test)
test_accuracy_SVC = accuracy_score(y_test, y_pred_SVC)
print(f"Test set accuracy for SVM: {test_accuracy_SVC*100:.1f} %")

Test set accuracy for SVM: 44.3 %


Тренируем модель K-ближайших соседей (K-Nearest Neighbors - KNN)

In [301]:
model_KNN = KNeighborsClassifier()
model_KNN.fit(X_train, y_train)

# Evaluate the model on the train set
y_pred_tr = model_KNN.predict(X_train)
train_accuracy_KNN = accuracy_score(y_train, y_pred_tr)
print(f"Train set accuracy for SVC: {train_accuracy_KNN*100:.1f} %")

Train set accuracy for SVC: 64.3 %


Оценка модели KNN на тестовой выборке

In [302]:
# Evaluate the model on the test set
y_pred_KNN = model_KNN.predict(X_test)
test_accuracy_KNN = accuracy_score(y_test, y_pred_KNN)
print(f"Test set accuracy for KNN: {test_accuracy_KNN*100:.1f} %")

Test set accuracy for KNN: 48.3 %


Методы ансамблей моделей

# Stacking

In [303]:
# Implementing Stacking, Bagging, Voting, and Boosting

# Stacking model (Logistic Regression)
from sklearn.linear_model import LogisticRegression

preds_cat = model_cat.predict(X_train)
preds_SVC = model_SVC.predict(X_train)
preds_KNN = model_KNN.predict(X_train)
    
# Combine predictions
stacked_pred = np.column_stack((preds_cat, preds_SVC, preds_KNN))
    
# Train stacking model (Logistic Regression)
stacking_model = LogisticRegression(solver='lbfgs', max_iter=10000)
stacking_model.fit(stacked_pred, y_train)
    
# obtain predictions for X_test from stacking model:
preds_cat_test = model_cat.predict(X_test)
preds_SVC_test = model_SVC.predict(X_test)
preds_KNN_test = model_KNN.predict(X_test)
    
# Combine predictions
stacked_pred_test = np.column_stack((preds_cat_test, preds_SVC_test, preds_KNN_test))
y_stacking = stacking_model.predict(stacked_pred_test)

# Bagging

In [304]:
# Bagging
from sklearn.ensemble import BaggingClassifier
bagging_model = BaggingClassifier(estimator=model_SVC, n_estimators=10)
bagging_model.fit(X_train, y_train)

BaggingClassifier(estimator=SVC())

# Voting

In [305]:
# Voting
# we know the predictions from the three models used
# So we can make voting by the averaging the predictions:
y_voting = np.round((y_pred_cat[:,0] + y_pred_SVC[:] + y_pred_KNN[:]) / 3)

# Boosting

In [306]:
# Boosting (using AdaBoost with SVM)
from sklearn.ensemble import AdaBoostClassifier
boosting_model = AdaBoostClassifier(estimator=model_cat, n_estimators=10)
boosting_model.fit(X_train, y_train)

c:\Users\User\anaconda3\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


AdaBoostClassifier(estimator=<catboost.core.CatBoostClassifier object at 0x000001A41E782BD0>,
                   n_estimators=10)

In [307]:
# Get stacking model accuracy on the test sample
stacked_accuracy = accuracy_score(y_test, y_stacking)
print(f"Stacking Model Accuracy: {stacked_accuracy*100:.1f} %")

Stacking Model Accuracy: 67.3 %


In [308]:
# Get bagging model accuracy on the test sample
bagging_accuracy = accuracy_score(y_test, bagging_model.predict(X_test))
print(f"Bagging Model Accuracy: {bagging_accuracy*100:.1f} %")

Bagging Model Accuracy: 44.0 %


In [309]:
# Get voting model accuracy on the test sample
voting_accuracy = accuracy_score(y_test, y_voting)
print(f"Voting Model Accuracy: {voting_accuracy*100:.1f} %")

Voting Model Accuracy: 56.2 %


In [310]:
# Get boosting model accuracy on the test sample
boosting_accuracy = accuracy_score(y_test, boosting_model.predict(X_test))
print(f"Boosting Model Accuracy: {boosting_accuracy*100:.1f} %")

Boosting Model Accuracy: 67.9 %


Выводы: <br>
На основе датасета winequality-white.csv, в котором приведены данные о качестве нескольких тысяч образцов вин и ряд их физико-химических параметров, было проведено исследование методов ансамблей моделей и их влияние на качество классификации вин на основе их измеренных параметров.
Вначале были созданы и обучены три модели мультиклассовой классификации, основанных на методе градиентного бустинга (aka CatBoost), методе опорных векторов (SVM) и методе К-ближайших соседей. Без дополнительного поиска наилучшего набора гиперпараметров (что не входило в нашу задачу на этот раз) для тестовой выборки были получены следующие параметры точности (accuracy) предсказания класса вина:
- CatBoost 67.3%
- Support Vector Machine 44.3%
- K-means 48.3%

Затем на основе этих трех обученных моделей были созданы ансамбли моделей. <br>
А именно, реализованы методы Стэкинга, Бэггинга, Голосования (Вотинга) и Бустинга.
Полученные точности составили:
- Стекинг 67.4%
- Бэггинг 44.0%
- Вотинг 56.2%
- Бустинг 67.9%

Лучшим из ансамблей оказался бустинг.
Можно сделать вывод, что удачно сконструированный ансамбль моделей  позволяет повысить точность решения задачи классификации, хотя и незначительно по сравнению с лучшим из использованных референсных методов, а именно CatBoost. <br>
Это объясняется тем, что сам CatBoost внутри себя использует бустинг, где последовательно рассчитываемые деревья используются для исправления ошибки предыдущих деревьев. 